# Small Experiment Data Analysis

In [33]:
import json
from six.moves import cPickle
import os
import statistics
from collections import defaultdict

from IPython.core.display import HTML 
from IPython.core.display import Image, display, display_pretty

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
"""
Load Coco data
"""
COCO_PATH = 'data/coco/'
COCO_ANNO_PATH = COCO_PATH + 'annotations/'

# load the data from captions
with open(COCO_ANNO_PATH + 'captions_train2014.json') as f:
    coco_captions = json.load(f)
# print len(coco_captions)
# with open(COCO_ANNO_PATH + 'captions_val2014.json') as f:
#     coco_captions =  dict(coco_captions.items() + json.load(f).items())
# print len(coco_captions)

# build the reverse dictionary, from img_id to captions, img_infos, and annotations
img_captions = {}
for img_info in coco_captions['images']:
    mid = int(img_info['id'])
    if not mid in img_captions:
        img_captions[mid] = {}
    img_captions[mid]['image'] = img_info

for cap_info in coco_captions['annotations']:
    mid = int(cap_info['image_id'])
    if not 'annotation' in img_captions[mid]:
        img_captions[mid]['annotation'] = []
        img_captions[mid]['captions'] = []
    img_captions[mid]['annotation'].append(cap_info)


In [44]:
IN_FILE_NAME  = 'scores_no_rel_texts_blackout.pkl'
OUT_FILE_NAME = 'no_rel_texts_img_ids'
IN_DIR        = 'input'
OUT_DIR       = 'output'

def pre_process_vis(vis, scores, l):
    """Since with large batch size, vis.json might contain duplicates. This function will purge out the duplicates.
    l is the number of image ids used to generate vis. After filtering, vis should have length 2l."""
    surplus = len(vis)-2*l
    return vis[:-surplus], scores[:-(surplus/2)]

def get_stat(data):
    if len(data)==0: return;
    print ""
    print "Total:\t%d;\t\tMean:\t\t%f;\tNonezero:\t%d"%(len(data), statistics.mean(data), len(filter(lambda x:x!=0, data)))
    print "Median:\t%f;\tMedian(H):\t%f;\tMedian(L):\t%f;"%(statistics.median(data), statistics.median_high(data), statistics.median_low(data))
    print "Max:\t%f;\tMin:\t\t%f;\tStd:\t\t%f"%(max(data), min(data),statistics.stdev(data))
    print "\n"
    
def look_up_image(title, idx, in_ids, vis, scores, out_file):
    print "[%d]%s\n"%(idx,title)
    print "\tScore:%s;\tImageId:%s\n"%(scores[idx], in_ids[idx])
    print "\tOriginal Caption:\n\t\t%s;\n\tAblation Caption:\n\t\t%s;"%(vis[idx*2+1]['caption'],vis[idx*2]['caption'])
    print "\tAnnotated Captions:\n"
    img_id = int(in_ids[idx])
    for i, note in enumerate(img_captions[img_id]['annotation']):
        print "\t\t%d. %s\n"%(i+1, str(note['caption']).strip())
    
    # Display images side by side: http://permalink.gmane.org/gmane.comp.python.ipython.devel/11073
    s = """<table>
        <tr>
        <th><img src="%s" style="max-width:400px" /></th>
        <th><img src="%s" style="max-width:400px" /></th>
        </tr></table>"""%(img_captions[img_id]['image']['coco_url'], "%s"%os.path.join("tmp_%s"%out_file, "%s_%s_ablt.jpg"%(str(idx).zfill(16),str(img_id))))
    t=HTML(s)
    display(t)

def get_expr_summary(in_file=IN_FILE_NAME, out_file=OUT_FILE_NAME, in_path=IN_DIR, out_path=OUT_DIR, num=3):
    in_ids  = cPickle.load(open(os.path.join(in_path, in_file)))
    scores  = cPickle.load(open(os.path.join(out_path, "scores_%s.pkl"%out_file)))
    vis     = json.load(open(os.path.join(out_path, 'vis_%s.json'%out_file)))
    vis, scores = pre_process_vis(vis, scores, len(in_ids))
#     print "Statistics for scores of gaussian filtered ablations:"
    get_stat(scores)

    scores_idx = zip(range(len(scores)), scores)
    sorted_scores_idx = sorted(scores_idx, key=lambda x: x[1])   # sort by score
    for idx, _ in sorted_scores_idx[:num]:
        look_up_image("Images with lowest scores:",idx, in_ids, vis, scores, out_file)



In [45]:
IN_FILE_NAME  = 'no_rel_texts_img_ids.pkl'
OUT_FILE_NAME = 'no_rel_texts_blackout'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME, num=3)


Total:	15980;		Mean:		0.056327;	Nonezero:	6696
Median:	0.000000;	Median(H):	0.000000;	Median(L):	0.000000;
Max:	1.000000;	Min:		0.000000;	Std:		0.094948


[0]Images with lowest scores:

	Score:0.0;	ImageId:392136

	Original Caption:
		a man with a hat and a hat on a clock;
	Ablation Caption:
		a group of people standing next to each other;
	Annotated Captions:

		1. A large bus and some people on the street.

		2. Several people are standing on the sidewalk as a bus goes by.

		3. Bus rushing by a group of people walking in a city.

		4. A double-decker bus moving down the street as people stand waiting.

		5. A group of people standing next to a yellow and blue double decker bus.



,


[2]Images with lowest scores:

	Score:0.0;	ImageId:71631

	Original Caption:
		a man and a woman standing next to each other;
	Ablation Caption:
		a truck is parked in a parking lot;
	Annotated Captions:

		1. Dining room table set for a casual meal, with flowers.

		2. A red table topped with four white place mats.

		3. there is a dining room table with a red cloth and a vase with roses

		4. a table with a red tablecloth and white placemats

		5. A small dinning table with all red napkins and a red table cloth .



,


[4]Images with lowest scores:

	Score:0.0;	ImageId:279108

	Original Caption:
		a desk with a laptop and a keyboard;
	Ablation Caption:
		a bus is driving down the street in a city;
	Annotated Captions:

		1. A woman feeding a man food from a spoon.

		2. A woman offering a man a taste of something in front of other people.

		3. A woman feeds a man a bite of food.

		4. a woman is feeding something to a man

		5. A woman spoon feeding an old man



,


In [10]:
IN_FILE_NAME  = 'rel_texts_img_ids.pkl'
OUT_FILE_NAME = 'rel_texts_blackout'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)


Total:	5600;		Mean:		0.074113;	Nonezero:	2767
Median:	0.000000;	Median(H):	0.000000;	Median(L):	0.000000;
Max:	1.000000;	Min:		0.000000;	Std:		0.113087


[2]Iamges with lowest scores:

	Score:0.0;	ImageId:257350

	Original Caption:
		a close up of an orange and a apple;
	Ablation Caption:
		a bus is parked in a parking lot;
	Annotated Captions:

		1. a group of people riding bikes stopped in front of a building

		2. A group of people on bicy les in front of a church.

		3. Bike riders on the corner outside of a church.

		4. Several children on bicycles outside a white church.

		5. Several people on bikes in front of a building.



,


[3]Iamges with lowest scores:

	Score:0.0;	ImageId:311914

	Original Caption:
		a lunch box with a variety of food;
	Ablation Caption:
		a man standing in front of a truck;
	Annotated Captions:

		1. A school bus parked with it's stop sign closed.

		2. A stop sign is on the side of a school bus.

		3. a bus sits stopped with a sign on the side of it

		4. Side of a school bus showing a stop sign.

		5. A view of a stop sign, on the side of a bus.



,


[12]Iamges with lowest scores:

	Score:0.0;	ImageId:462559

	Original Caption:
		a white van parked in front of a building;
	Ablation Caption:
		a street sign on the side of the road;
	Annotated Captions:

		1. The sign tells motorists how to proceed on the street.

		2. A stop light is topped with a do not enter warning.

		3. A traffic light sitting below a do not enter sign.

		4. A stop light with a one way, do not enter sign above it.

		5. A large sign over a black stop light.



,


In [66]:
IN_FILE_NAME  = 'large_text_img_ids.pkl'
OUT_FILE_NAME = 'large_text_gaussian'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)

Statistics for scores of gaussian filtered ablations:

Total:	683;		Mean:		0.818877;	Nonezero:	681
Median:	1.000000;	Median(H):	1.000000;	Median(L):	1.000000;
Max:	1.000000;	Min:		0.000000;	Std:		0.279659


[378]Zero Image

	Score:0.0;	ImageId:247708

	Original Caption:
		a person holding a cell phone in their hand;
	Ablation Caption:
		a laptop computer sitting on top of a table;
	Annotated Captions:

		1. a person operating a cell phone near a laptop

		2. A laptop, smartphone, and iPad all on a table

		3. A person is calling someone on his phone regarding information on his laptop.

		4. A person working on a computer who is holding an iPhone.

		5. A person multitasking by using a laptop, phone, and tablet.



,


[480]Zero Image

	Score:0.0;	ImageId:546211

	Original Caption:
		a man is skiing down a snowy hill;
	Ablation Caption:
		a group of people standing on top of a snow covered slope;
	Annotated Captions:

		1. A skier is standing beside a rack of skis and snowboards.

		2. A skier looks at a wooden rack of skis at the bottom of a slope.

		3. A person on some skis in the snow.

		4. Skiers coming down a slope on a sunny day.

		5. A man skiing next to a ski rack.



,


[150]Zero Image

	Score:0.0833333333333;	ImageId:132883

	Original Caption:
		a street sign on a pole on a city street;
	Ablation Caption:
		a red and white building with a clock on the side of it;
	Annotated Captions:

		1. A red and white street sign sitting in front of a church.

		2. A sign with spray paint all over it .

		3. The sign describes the hours of the attraction.

		4. A red and white sign is all scribbled on.

		5. Vandalism on a sign with words that are not in English.



,


In [67]:
IN_FILE_NAME  = 'large_text_img_ids.pkl'
OUT_FILE_NAME = 'large_text_blackout'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)

Statistics for scores of gaussian filtered ablations:

Total:	683;		Mean:		0.521510;	Nonezero:	630
Median:	0.500000;	Median(H):	0.500000;	Median(L):	0.500000;
Max:	1.000000;	Min:		0.000000;	Std:		0.352480


[23]Zero Image

	Score:0.0;	ImageId:330436

	Original Caption:
		a man is doing a trick on a skateboard;
	Ablation Caption:
		a person jumping a skate board in the air;
	Annotated Captions:

		1. a person jumping a skate board in the air

		2. This skateboarder is riding the side of his skateboard in the air.

		3. The young man is riding his skateboard practicing his moves.

		4. A skateboarder is airborne in front of a wall covered in graffiti.

		5. A person jumping in the air on a skateboard.



,


[27]Zero Image

	Score:0.0;	ImageId:375294

	Original Caption:
		a couple of cell phones sitting on top of a table;
	Ablation Caption:
		a person is holding a small piece of paper;
	Annotated Captions:

		1. there are 2 tablets, 1 e-reader, and an iphone sitting on the table.

		2. a table that has a bunch of ipads on it

		3. some different tablets cellphone and a piece of pizza

		4. Tablets and a cell phone on a table beside a plate of pizza.

		5. Some iPads and phone are on a table with pizza.



,


[33]Zero Image

	Score:0.0;	ImageId:172316

	Original Caption:
		a young boy swinging a baseball bat at a ball;
	Ablation Caption:
		a man standing in front of a wall with a cell phone;
	Annotated Captions:

		1. A boy is swinging at a ball during a baseball game.

		2. A boy that is hitting a ball with a baseball bat.

		3. A person with a bat and helmet hits a ball.

		4. a boy in a yellow and white uniform hitting a baseball

		5. A young baseball player is hitting the ball.



,


In [50]:
vis = json.load(open(os.path.join(OUT_DIR, 'vis_%s.json'%OUT_FILE_NAME)))
scores  = cPickle.load(open(os.path.join(OUT_DIR, "scores_%s.pkl"%OUT_FILE_NAME)))
in_ids  = cPickle.load(open(os.path.join(IN_DIR, IN_FILE_NAME)))

In [56]:
print len(in_ids), len(vis), in_ids[0], len(scores)
print OUT_FILE_NAME, IN_FILE_NAME 

vp, sp = pre_process_vis(vis, scores, len(in_ids))
print len(vp), len(sp)

from semantic_dist import *
ablated = [d['caption'] for d in vp[::2]]
original = [d['caption'] for d in vp[1::2]]
stoplist = set('for a of the and to in its his her'.split())
ablated, original = pre_process(ablated, ignore=stoplist),pre_process(original, ignore=stoplist)
new_sp = map(lambda x: calc_inter_union(*x), zip(ablated, original))

assert len(sp) == len(new_sp)
for i in range(len(sp)):
    assert sp[i]==new_sp[i]
#figuring out what is duplicated
# from collections import defaultdict
# s = defaultdict(int)
# for d in vp:
#     s[int(d['image_id'])] += 1
# print len(s), len(l)
# print l
# print vis[0] == vis[-40]

15980 32000 392136 16000
no_rel_texts_blackout no_rel_texts_img_ids.pkl
31960 15980


In [62]:
for idx, d in enumerate(vp):
    if 'a group of people standing in front of a bus' in d['caption']:
        print idx, d
print len(in_ids), len(set(in_ids)), 392136 in in_ids

245 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'246'}
899 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'900'}
913 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'914'}
1565 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'1566'}
2314 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'2315'}
3230 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'3231'}
3790 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'3791'}
4213 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'4214'}
4318 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'4319'}
4583 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'4584'}
4647 {u'caption': u'a group of people standing in front of a bus', u'image_id': u'4648'}
4666 {u'caption': u'a group